In [1]:
from music21 import *
from comp2 import *

In [2]:
melody_in: stream.Stream = parse_midi("example_inputs/eyeofthetiger.mid")

print("Original melody:")
melody_in.show('midi');

Original melody:


In [3]:
print("Simple triads:")
simple_triads = get_triads(melody_in)

Simple triads:
['iii', 'ii5#2', 'ii5#2', 'ii5#2', 'iii', 'iii', 'ii5#2', 'iii', 'iii', 'ii5#2', 'ii5#2', 'ii5#2', 'iii', 'iii', 'ii5#2', 'iii', 'vi+64', 'I', 'vi+64', 'iii', 'vi+64', 'I', 'vi+64', 'vi', 'ii5#2']


In [4]:
out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, simple_triads)

out_score.show('midi')

In [5]:
markov_table = generate_markov_table()
substituted_chords = substitute_chords(simple_triads, markov_table)
for i in range(5):
    substituted_chords = substitute_chords(substituted_chords, markov_table)

out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, substituted_chords)

out_score.show('midi')

Total number of states: 2744

new chords:
['iii2', 'ii76', 'ii2', 'ii2', 'iii2', 'iii2', 'vi2', 'iii', 'iii', 'ii2', 'iisus4', 'ii7', 'iii7', 'iiisus4', 'ii76', 'iii2', 'vi2', 'ii2', 'vi7', 'ii2', 'vi2', 'ii76', 'vi2', 'vi7', 'ii7']

new chords:
['iii2', 'vi', 'vi76', 'vi76', 'iii2', 'iii', 'vi2', 'II2', 'vi2', 'vi2', 'ii2', 'IIsus4', 'iii2', 'vi76', 'ii7', 'iii', 'vi7', 'VI7', 'ii2', 'vi76', 'vi', 'vii', 'vi2', 'vi2', 'vi2']

new chords:
['iii2', 'vi6', 'VI2', 'vi2', 'iii2', 'iii2', 'vi2', 'V7', 'V2', 'vii6', 'vi2', 'v2', 'iiisus4', 'vi7', 'ii76', 'iii2', 'i', 'VII7', 'vi7', 'ii2', 'vi2', 'vii2', 'vi2', 'vi2', 'vi2']

new chords:
['iii2', 'vi2', 'ii7', 'vi7', 'iii2', 'iii', 'vi', 'ii2', 'iii', 'vii7', 'vi2', 'iiisus4', 'vi2', 'ii6', 'vii7', 'iiisus4', 'vi2', 'VII', 'vi', 'vi76', 'vi2', 'vii7', 'vi2', 'ii2', 'vi2']

new chords:
['iii2', 'vi2', 'i7', 'vi7', 'ii7', 'iii', 'vi', 'III7', 'iii', 'viisus4', 'vi', 'vi7', 'vi', 'ii7', 'v2', 'vi76', 'vi', 'VII2', 'ii2', 'vi2', 'Isus4', 'V2', 'v